In [1]:
import warnings
warnings.filterwarnings('ignore')
import time

- Importation des modules
- Chargement du fichier de données et retraitement du fichier de données
- Transformation du DataFrame des MetaDonnée en Matrice

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from numba import jit
from keras.layers import LSTM, Activation, Dropout, Dense, Input, Conv1D, MaxPooling1D, GlobalMaxPooling1D
from keras.layers import Embedding
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras_preprocessing.sequence import pad_sequences
import keras

df = pd.read_csv("Data_Satisfaction_retraitement3.csv")

df = df.dropna(axis = 0, how = "all", subset =["Commentaires_reduit"]).reset_index()

y = df["star"]
#Com_Matrix = df["Commentaires_reduit"].to_numpy()

- intégration des commentaire dans une liste
- décalage des valeurs de y pour y inclure 0

In [3]:
liste_commentaires = []
for i in range(len(df.Commentaires_reduit)):
      liste_commentaires.append(df.Commentaires_reduit[i])
y = y.replace(to_replace = [1,2,3,4,5], value = [0,1,2,3,4])

création d'un ensemble de test et d'un ensemble d'entrainement

In [4]:
X_train, X_test,Y_train, Y_test = train_test_split(liste_commentaires, y, test_size=0.2, random_state = 45)

Tokenization des mots contenue dans X_train et affichage de l'index de chaque mots

In [5]:
tokenizer = Tokenizer(num_words=50000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

Création d'une fonction permant la lecture du fichier Glove pre-entrainé et la création d'un dictionnaire contenant l'ensemble des mots du fichier ainsi que leur vecteur associé.

In [6]:
def read_glove_vector(glove_vec):
    f = open(glove_vec, 'r', encoding='UTF-8')
    word_to_vec_map = {}
    for line in f:
        w_line = line.split()
        curr_word = w_line[0]
        word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

    return word_to_vec_map

lecture du fichier Glove , et determination d'une taille de commentaire maximale

In [7]:
word_to_vec_map = read_glove_vector('glove.twitter.27B.50d.txt')

maxLen = 200

Création d'une matrice d'emmbeding à l'aide du dictionnaire Glove crée plus haut. Si un mot n'est pas présent dans le dictionnaire, un vecteur nul est affecté à la place.

In [8]:
vocab_len = len(words_to_index)+1
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)        

In [ ]:
print(vocab_len)
arr = emb_matrix
print(f"number of zeros: {(arr.size - np.count_nonzero(arr))/50}")
print(arr.shape)

Definition d'une fonction permettant de crée un model en fonction de la taille générique des commentaires:

In [28]:
def commentaires_rating(input_shape):
    X_indices = Input(input_shape)
    embeddings = embedding_layer(X_indices)
    X = LSTM(128)(embeddings)
    X = Dropout(0.6)(X)
    X = LSTM(128)(embeddings)
    X = Dropout(0.6)(X)
    X = Dense(64, activation='relu')(X)
    X = Dense(5, activation='softmax')(X)
    model = Model(inputs=X_indices, outputs=X)
    return model

Padding des données d'entrainement, c'est à dire redimmensionnement de la taille des commentaires pour que tous aient une longueur maximale correspondant à la valeur maxLen définit plus haut.

In [10]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

Création de deux fonctions de call back : l'une pour affiné progressivement le learning rate, et l'autre pour arreter le model lorsque plus aucune augmentation significative de la précision est enregistré.

In [11]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
reducelr = ReduceLROnPlateau(monitor = 'accuracy',
                        min_delta = 0.001,
                        patience = 2,
                        factor = 0.5, 
                        cooldown = 2,
                        verbose = 1)


from tensorflow.keras.callbacks import EarlyStopping 
earlystop = EarlyStopping(monitor = 'accuracy',
                    min_delta = 0.0001,
                    patience = 4,
                    verbose = 1,
                    restore_best_weights = True)

Création du model adapté à des commentaires de taille (nombre de caractères) Maxlen.

In [29]:
model1 = commentaires_rating((maxLen,))

Configuration de l'optimizer, compilation et entrainement du model.

In [30]:
adam = keras.optimizers.Adam(learning_rate = 1)
model1.compile(optimizer=adam, loss="sparse_categorical_crossentropy", metrics=['accuracy'], )

m1_history = model1.fit(X_train_indices, Y_train, batch_size=30, epochs=15, validation_split = 0.2, callbacks= [reducelr,earlystop])

Epoch 1/15
355/355 [==============================] - 33s 90ms/step - loss: 2.8033 - accuracy: 0.3287 - val_loss: 1.5065 - val_accuracy: 0.2874 - lr: 1.0000
Epoch 2/15
355/355 [==============================] - 31s 87ms/step - loss: 1.5305 - accuracy: 0.3243 - val_loss: 1.5025 - val_accuracy: 0.3621 - lr: 1.0000
Epoch 3/15
355/355 [==============================] - ETA: 0s - loss: 1.5174 - accuracy: 0.3202
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.5.
355/355 [==============================] - 31s 86ms/step - loss: 1.5174 - accuracy: 0.3202 - val_loss: 1.4993 - val_accuracy: 0.3621 - lr: 1.0000
Epoch 4/15
355/355 [==============================] - 29s 83ms/step - loss: 1.5068 - accuracy: 0.3566 - val_loss: 1.4993 - val_accuracy: 0.3621 - lr: 0.0000e+00
Epoch 5/15
355/355 [==============================] - 30s 85ms/step - loss: 1.5068 - accuracy: 0.3566 - val_loss: 1.4993 - val_accuracy: 0.3621 - lr: 0.0000e+00
Epoch 6/15
355/355 [==============================] - 30s 85ms/s

Au bout de 10 époch , on obtient une valeur de validation accuracy de 0.3621, sans aucune augmentation signigicative entre les épochs.

Le modèle définit ne se montre pas efficace pour prédire les notes en fonction des commentaires. 

Essayons avec un autre model, basé sur les matrice convolutives:

In [22]:
def conv1d_model(input_shape):
    X_indices = Input(input_shape)
    embeddings = embedding_layer(X_indices)
    X = Conv1D(512,3,activation='relu')(embeddings)
    X = MaxPooling1D(3)(X)
    X = Conv1D(256,3,activation='relu')(X)
    X = MaxPooling1D(3)(X)
    X = Conv1D(256,3,activation='relu')(X)
    X = Dropout(0.8)(X)
    X = MaxPooling1D(3)(X)
    X = GlobalMaxPooling1D()(X)
    X = Dense(256, activation='relu')(X)
    X = Dense(5, activation='softmax')(X)
    model = Model(inputs=X_indices, outputs=X)
    return model

In [23]:
model2 = conv1d_model((maxLen,))

In [24]:
adam = keras.optimizers.Adam(learning_rate = 1)
model2.compile(optimizer=adam, loss="sparse_categorical_crossentropy", metrics=['accuracy'], )

m2_history = model2.fit(X_train_indices, Y_train, batch_size=30, epochs=15, validation_split = 0.2, callbacks= [reducelr,earlystop])

Epoch 1/15
355/355 [==============================] - 19s 52ms/step - loss: 34530696.0000 - accuracy: 0.3310 - val_loss: 1.4779 - val_accuracy: 0.2874 - lr: 1.0000
Epoch 2/15
355/355 [==============================] - 17s 49ms/step - loss: 1.5158 - accuracy: 0.3298 - val_loss: 1.5271 - val_accuracy: 0.2874 - lr: 1.0000
Epoch 3/15
355/355 [==============================] - ETA: 0s - loss: 1.5171 - accuracy: 0.3159
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.5.
355/355 [==============================] - 18s 52ms/step - loss: 1.5171 - accuracy: 0.3159 - val_loss: 1.4573 - val_accuracy: 0.3621 - lr: 1.0000
Epoch 4/15
355/355 [==============================] - 18s 50ms/step - loss: 1.4708 - accuracy: 0.3566 - val_loss: 1.4573 - val_accuracy: 0.3621 - lr: 0.0000e+00
Epoch 5/15
355/355 [==============================] - 17s 49ms/step - loss: 1.4708 - accuracy: 0.3566 - val_loss: 1.4573 - val_accuracy: 0.3621 - lr: 0.0000e+00
Epoch 6/15
355/355 [==============================] - 17s

Au bout de 10 époch , on obtient une valeur de validation accuracy de 0.3621, sans aucune augmentation significative entre les épochs.

Ce modèle ne se montre pas plus éfficace que le précédent pour prédire les notes en fonction des commentaires. 

In [25]:
def Dense_model(input_shape):
    X_indices = Input(input_shape)
    embeddings = embedding_layer(X_indices)
    X = GlobalMaxPooling1D()(embeddings)
    X = Dense(32, activation='relu')(X)
    X = Dense(5, activation='softmax')(X)
    model = Model(inputs=X_indices, outputs=X)
    return model

In [26]:
model3 = Dense_model((maxLen,))
adam = keras.optimizers.Adam(learning_rate = 1)
model3.compile(optimizer=adam, loss="sparse_categorical_crossentropy", metrics=['accuracy'], )

m3_history = model3.fit(X_train_indices, Y_train, batch_size=30, epochs=15, validation_split = 0.2, callbacks= [reducelr,earlystop])

Epoch 1/15
355/355 [==============================] - 1s 2ms/step - loss: 3.2325 - accuracy: 0.3193 - val_loss: 1.5060 - val_accuracy: 0.3621 - lr: 1.0000
Epoch 2/15
355/355 [==============================] - 1s 2ms/step - loss: 1.5267 - accuracy: 0.3208 - val_loss: 1.5166 - val_accuracy: 0.3621 - lr: 1.0000
Epoch 3/15
355/355 [==============================] - 1s 2ms/step - loss: 1.5222 - accuracy: 0.3212 - val_loss: 1.5200 - val_accuracy: 0.3621 - lr: 1.0000
Epoch 4/15
355/355 [==============================] - 1s 2ms/step - loss: 1.5168 - accuracy: 0.3320 - val_loss: 1.4902 - val_accuracy: 0.3621 - lr: 1.0000
Epoch 5/15
355/355 [==============================] - 1s 2ms/step - loss: 1.5338 - accuracy: 0.3221 - val_loss: 1.5314 - val_accuracy: 0.3621 - lr: 1.0000
Epoch 6/15
348/355 [============================>.] - ETA: 0s - loss: 1.5161 - accuracy: 0.3205
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.5.
355/355 [==============================] - 1s 2ms/step - loss: 1.5177 -

Au bout de 10 époch , on obtient une valeur maximal de validation accuracy de 0.3621, sans aucune augmentation significative entre les épochs.

Ce modèle ne se montre pas plus éfficace que le précédent pour prédire les notes en fonction des commentaires. 